### Comparing Models and Vectorization Strategies for Text Classification

This try-it focuses on weighing the positives and negatives of different estimators and vectorization strategies for a text classification problem.  In order to consider each of these components, you should make use of the `Pipeline` and `GridSearchCV` objects in scikitlearn to try different combinations of vectorizers with different estimators.  For each of these, you also want to use the `.cv_results_` to examine the time for the estimator to fit the data.

### The Data

The dataset below is from [kaggle]() and contains a dataset named the "ColBert Dataset" created for this [paper](https://arxiv.org/pdf/2004.12765.pdf).  You are to use the text column to classify whether or not the text was humorous.  It is loaded and displayed below.

**Note:** The original dataset contains 200K rows of data. It is best to try to use the full dtaset. If the original dataset is too large for your computer, please use the 'dataset-minimal.csv', which has been reduced to 100K.

In [5]:
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB

from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords

In [2]:
df = pd.read_csv('text_data/dataset.csv')

In [3]:
df.head()

,text,humor
0,"Joe biden rules out 2020 bid: 'guys, i'm not r...",False
1,Watch: darvish gave hitter whiplash with slow ...,False
2,What do you call a turtle without its shell? d...,True
3,5 reasons the 2016 election feels so personal,False
4,"Pasco police shot mexican migrant from behind,...",False


#### Task


**Text preprocessing:** As a pre-processing step, perform both `stemming` and `lemmatizing` to normalize your text before classifying. For each technique use both the `CountVectorize`r and `TfidifVectorizer` and use options for stop words and max features to prepare the text data for your estimator.

**Classification:** Once you have prepared the text data with stemming lemmatizing techniques, consider `LogisticRegression`, `DecisionTreeClassifier`, and `MultinomialNB` as classification algorithms for the data. Compare their performance in terms of accuracy and speed.

Share the results of your best classifier in the form of a table with the best version of each estimator, a dictionary of the best parameters and the best score.

In [35]:
import nltk
nltk.download('wordnet')
  
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
from sklearn.base import BaseEstimator, TransformerMixin

def stem_text(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

def lemmatize_text(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

class StemmingTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.apply(stem_text)

class LemmatizationTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.apply(lemmatize_text)
    
tokenizers = {
    "Stemmer": StemmingTransformer(),
    "Lemmatizer": LemmatizationTransformer(),
}

vectorizers = {
    "CountVectorizer": CountVectorizer(),
    "TfidfVectorizer": TfidfVectorizer(),
}

[nltk_data] Downloading package wordnet to /home/henrique/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [36]:
results = {}

param_grid = {
    # "vectorizer": [CountVectorizer(max_features=500, stop_words=None), TfidfVectorizer(max_features=500, stop_words=None)],
    "classifier": [
        LogisticRegression(max_iter=1000),
        DecisionTreeClassifier(),
        MultinomialNB(),
    ],
}


for tkname, tk in tokenizers.items():
    X_transformed = tk.fit_transform(df["text"])
    X_train, X_test, y_train, y_test = train_test_split(
        X_transformed, df["humor"], test_size=0.3, random_state=42
    )
    for vcname, vc in vectorizers.items():
        out = {
            "tokenizer": tkname,
            "vectorizer": vcname,
        }
        pipeline = Pipeline([("vectorizer", vc), ("classifier", None)])
        gridcv = GridSearchCV(
            pipeline, param_grid, cv=3, scoring="accuracy", n_jobs=-1, return_train_score=True, verbose=4,
        ).fit(X_train, y_train)

stemmed_X = StemmingTransformer().fit_transform(df["text"])
lemmatized_X = LemmatizationTransformer().fit_transform(df["text"])

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV 1/3] END classifier=LogisticRegression(max_iter=1000);, score=(train=0.956, test=0.926) total time=   4.4s
[CV 2/3] END classifier=LogisticRegression(max_iter=1000);, score=(train=0.957, test=0.925) total time=   4.9s
[CV 3/3] END classifier=LogisticRegression(max_iter=1000);, score=(train=0.957, test=0.924) total time=   4.8s
[CV 1/3] END classifier=MultinomialNB();, score=(train=0.927, test=0.914) total time=   2.4s
[CV 2/3] END classifier=MultinomialNB();, score=(train=0.927, test=0.913) total time=   3.3s
[CV 3/3] END classifier=MultinomialNB();, score=(train=0.928, test=0.913) total time=   2.8s
[CV 1/3] END classifier=DecisionTreeClassifier();, score=(train=1.000, test=0.865) total time=  59.3s
[CV 2/3] END classifier=DecisionTreeClassifier();, score=(train=1.000, test=0.865) total time=  59.5s
[CV 3/3] END classifier=DecisionTreeClassifier();, score=(train=1.000, test=0.863) total time= 1.0min
Fitting 3 folds for eac